## RSA - The Project

Name: Connor Brown 

**The Prompt:**

Your boss has been long intrigued by the RSA algorithm and has asked you to create a tutorial RSA walk through and exploration project to help the boss and others in the company to understand the basic structures and mathematics of an RSA implementation.

This project will:
1. Implement the algorithm for breaking codes with a given basic structure. 
2. Explain the steps 
4. Implement a custom feature using previously written code.

<hr />

# Table of Contents

## 1. Introduction to RSA Project


## 2. Code Package: FME
        * Convert_Binary_String 
        * FME

## 3. Code Package: Key Generation using Euclidean Algorithms
        * Euclidean Algorithm
        * Extended EA
        * Find_Public_Key_e
        * Find_Private_Key_d

## 4. Code Package: En/Decode Messages
        * Convert_Text
        * Convert_Num
        * Encode
        * Decode


## 5. Code Demo

## 6. Code Exchange Results

## 7. Narrative

## 8. Code Breaking

## 9. Code Breaking: Complete Examples

## 10. Custom Feature


    


## 1. Introduction to RSA Project


The Rivest-Shamir-Adleman algorithm, or RSA for short, is the most common way for encrypting and exchanging information securely. In this project, I will be building RSA from scratch, using number theory and Python programming. RSA relvolves around public and private keys. A user sends a public key to another user, that user encrypts their information with the public key, and sends it back to the original user, who decrypts the information with the private key. To get these keys, we need two different prime numbers p and q. Their product n as well as phi function (p - 1)(q - 1) are used in determining the keys. p and q are taken as inputs to FindPublicKey(). n and phi_n (the phi function) are calculated, and then an intial value for e, the public key, is determined using a random number generator. e must fulfill three conditions: be coprime to phi_n and not equal to either p or q. Then, a private key d is determined by using the Extended Euclidean Algorithm to get the modular inverse of e mod phi. e and d are used to convert the ASCII characters in a message into numbers that have no apparent relationship to the original ASCII values, then convert the unreadable number list back into a comprehensible message. In this project, I will be showing the Python implementation of each of these steps. The end result will be a program with a custom main function that users can use to read, encrypt and decrypt messages.

## 2. Code Package FME

In [62]:
def Convert_Binary_String(_int):
    """
    Converts an integer to a string of its binary expansion. May or may not be needed.
    
    For example:
    _int = 345
    bits = 101011001
    """
    
    return bin(_int)[2:]

In [63]:
def FME(b, n, m):
    """
    Using the fast modular exponentiation algorithm, returns b**n mod m.
    """
    
    # initial values
    result = 1
    square = b
    
    while (n > 0):
        # convert n to binary, start from least significant digit
        k = n % 2 
        # add to result if binary digit is 1
        if k == 1:
            result = (result * square) % m
        # advance exponent base
        square = (square * square) % m
        # advance to next digit in n while reducing n
        n //= 2
        
    return result

## 3. Code Package: Key Generation using Euclidean Algorithms

In [64]:
def Euclidean_Alg(a, b):
    """
    1. Calculate the Greatest Common Divisor of a and b.
    
    2. This version should have only positive inputs and outputs.
    
    3. The function must return a single integer ('x') which is
    the gcd of a and b.
    """
    
    # check inputs for correct type and value
    if (type(a) != int) or (type(b) != int):
        raise TypeError("Inputs must be positive integers.")
    if (a <= 0) or (b <= 0):
        raise ValueError("Inputs must be positive integers.")
    
    # swap inputs if a is not greater than b
    if (b > a):
        temp = a
        a = b
        b = temp
    
    # calculate gcd
    while (b > 0):
        # find remainder of a / b
        k = a % b
        # advance a to next smallest number
        a = b
        # advance b to remainder
        b = k
    
    # gcd equal to final value of a
    x = a
    
    return x

In [65]:
def EEA(a, b):
    """ 
    Extended Euclidean Algorithm - This version will return both: 
    1. the GCD of a, b 
    2. Bezout's coefficients for representing the GCD as a linear combination.
    """
        
    # check inputs for correct type and value - must be positive integers
    if (type(a) != int) or (type(b) != int):
        raise TypeError("Inputs must be positive integers.")
    if (a <= 0) or (b <= 0):
        raise ValueError("Inputs must be positive integers.")
    
    # swap inputs if a is not greater than b
    isSwapped = b > a
    if (isSwapped):
        temp = a
        a = b
        b = temp
        
    # initial values of a and b
    a0 = a
    b0 = b
    
    # initial values of Bezout coefficients
    s1, t1 = (1, 0)
    s2, t2 = (0, 1)
    
    # calculate gcd and Bezout coefficients
    while (b > 0):
        # update a and b using new Bezout coefficients
        a = (s1 * a0) + (t1 * b0)
        b = (s2 * a0) + (t2 * b0)
        
        # find remainder of a / b
        k = a % b
        # find integer quotient of a / b
        q = a // b
        
        # advance a to next smallest number
        a = b
        # advance b to remainder
        b = k
        
        # calculate Bezout coefficients for next smallest number
        s1_p, t1_p = (s2, t2)
        s2_p, t2_p = (s1 - (q * s2), t1 - (q * t2))
        
        # update actual Bezout coefficients
        s1, t1 = (s1_p, t1_p)
        s2, t2 = (s2_p, t2_p)
        
    # gcd equal to final value of a
    gcd = a
    
    # reflect original order of a and b
    if isSwapped:
        s1_final = t1
        t1_final = s1
    else:
        s1_final = s1
        t1_final = t1
        
    # ensure that s1_final is positive using modular inverse trick
    while (s1_final < 0):
        if isSwapped:
            s1_final += a0     # add a0
        else:
            s1_final += b0     # add b0
        t1_final = (gcd - (s1_final * a0)) // b0
        
    
    return gcd, (s1_final, t1_final)

In [66]:
def Find_Public_Key_e(p, q):
    """
    Inputs: 2 prime numbers, p and q.

    Uses the previously defined Euclid_Alg function.
    
    Returns 2 elements as follows:
    public key: n
    public key: e  
    """
    
    # for getting quality random numbers
    from secrets import randbelow
    
    # calculate n and phi(n)
    n = p * q
    phi_n = (p - 1) * (q - 1)
    
    # calculate initial value for e
    e = randbelow(10000)
    
    # update e value until e is relatively prime to phi_n and not equal to p or q
    while (Euclidean_Alg(phi_n, e) != 1 or e == p or e == q):
        e = randbelow(10000)
        
    return (n, e)
    

In [67]:
def Find_Private_Key_d(e, p, q):
    """
    Inputs: public key e and two prime numbers, p and q.
    
    This will use the Extended Euclidean Algorithm (EEA) function.
    
    Returns the following:
        d: the decryption component.
    """
    
    # calculate n and phi(n)
    n = p * q
    phi_n = (p - 1) * (q - 1)
    
    # d is modular inverse of e mod phi_n - calculate using EEA of (e, phi_n)
    gcd, (s1, t1) = EEA(e, phi_n)
    
    # d is equal to Bezout coefficient s1
    d = s1
    
    return d

## 4. Code Package: En/Decode Messages

In [68]:
def Convert_Text(_string):
    """
    Takes in a simple string such as "hello" and outputs the corresponding
    standard list of integers (ascii) for each letter in the string.

    For example:
    _string = hello
    integer_list = [104, 101, 108, 108, 111]
    """
    integer_list = []
    
    # add converted characters to list
    for char in _string:
        integer_list.append(ord(char))
        
    return integer_list

In [69]:
def Convert_Num(_list):
    """
    Takes in a list of integers and outputs the corresponding string (ascii).
    
    For example:
    _list = [104, 101, 108, 108, 111]
    _string = hello
    """
    _string = ''
    
    # update string with converted integers
    for i in _list:
        _string += chr(i)
        
    return _string

In [70]:
def Encode(n, e, message):
    """
    First, convert a string message into a list of integers using Convert_Text.
    
    Then, encode each of these integers using n and e and
    return the encoded cipher_text.
    """
    
    # get unencoded list
    int_list = Convert_Text(message)
    
    # initialize encoded list
    cipher_text = []
    
    # encode items in int_list using FME, add to cipher_text
    for m in int_list:
        m_encrypted = FME(m, e, n)
        cipher_text.append(m_encrypted)
    
    return cipher_text


In [71]:
def Decode(n, d, cipher_text):
    """
    First, decrypt each integer in cipher_text using n and d.
    
    Then, recover the original message as a string using Convert_Num.
    
    """
    
    # initialize decrypted message
    message = ''
    
    # decrypt cipher text list using FME
    decrypted_cipher_text = [FME(c, d, n) for c in cipher_text]
    
    # convert decrypted list to string
    message = Convert_Num(decrypted_cipher_text)

    return message

### 5. Code Demo

**Generating Keys**

First, pass two different prime numbers p and q to FindPublicKey(), and extract the return values n_val and e_val (the public keys):

In [72]:
p = 29
q = 347
n_val, e_val = Find_Public_Key_e(p, q)

Next, pass e_val, p and q to FindPrivateKey(), and extract the return value d (the private key):

In [73]:
d_val = Find_Private_Key_d(e_val, p, q)

*Note:* e must be relatively prime to phi ((p - 1)(q - 1)), and d must be a modular inverse of e mod phi. This is ensured by using the Euclidean Algorithm (Euclidean_Alg()) and the Extended Eucliean Algorithm (EEA()) helper functions inside of the public and private key functions. The standard Euclidean Algorithm checks that the GCD of e and phi is 1, proving they are relatively prime, and the Extended Euclidean Algorithm both checks coprimality AND finds a d such that it is a modular inverse of e mod phi. These conditions are critical to the encoding and decoding process that make RSA work. The public and private keys must be correctly paired, otherwise the encryption and decryption will not function correctly.

**Encoding Messages**

Now on to the fun part. Encode the characters using Encode(), making sure to pass in the public keys n_val and e_val determined earlier:

In [74]:
message = "Hello, World!"
encoded_text_list = Encode(n_val, e_val, message)

*Note:* Both Encode() and Decode() use Fast Modular Exponentiation (FME) to calculate the mod of each item in the number sequence. RSA, at full capacity, uses huge numbers (~100 digits) that take a very long time to process. Python's built-in modulo operator (%) does not use FME, meaning it has much longer runtimes as the numbers get bigger. The custom FME() function I wrote earlier and use inside of Encode() and Decode() is lightning-fast and better suited to the large numbers involved. Here's my encoded message:

In [75]:
print(encoded_text_list)

[5549, 9435, 2811, 2811, 9867, 2165, 47, 3799, 9867, 9747, 2811, 5893, 270]


**Decoding Messages**

Decode the number list using Decode(), making sure to pass in the n_val and private key d_val determined earlier, and see the result:

In [76]:
decoded_text = Decode(n_val, d_val, encoded_text_list)
print(decoded_text)

Hello, World!


### 6. Code Exchange Results

In [77]:
print()

# Question - by Patrick B.
encoded_message = [3255, 3531, 2178, 151, 3995, 3099, 2178, 485, 3995, 3099, 5370, 5439, 2178, 5319, 1116, 461, 245, 2178, 1116, 2178, 3258, 3099, 962, 245, 2497, 962, 3995, 882, 245, 2497, 347, 2178, 882, 5319, 1116, 3641, 2178, 882, 3995, 3099, 5370, 5439, 2178, 523, 3641, 2178, 2209, 245, 4313]
decoded_message = Decode(5609, 3971, encoded_message)
print(decoded_message)

# Answer to Question - Connor Brown
string = "I am stuck between unlimited physical regeneration and line-of-sight teleportation. I want to both be healthy and go all kinds of places!"
encoded_message = Encode(5609, 11, string)
print()
print("Cipher =", encoded_message)

print()
#################################

# Question - by Patricia V.
encoded_message = [69475, 24171, 42354, 43002, 7640, 36510, 11573, 45234, 34949, 35674, 34949, 7640, 34949, 44246, 69933, 60294, 34949, 42354, 52635, 11573, 45234, 42354, 7640, 7640, 69933, 12397, 75964, 33126, 53207, 11573, 77341, 24171, 69933, 60294, 11573, 12397, 34949, 33126, 45234, 33126, 11573, 42354, 44617, 11573, 8347, 33126, 36510, 34949, 69933, 11573, 38127, 69584, 42354, 42354, 76473, 53207, 11573, 8347, 42354, 35674, 34949, 33126, 53207, 11573, 69933, 7640, 69584, 43002, 8347, 53207, 11573, 33126, 60294, 45234, 24596, 15300, 11573, 77341, 42354, 43002, 7640, 36510, 11573, 12264, 42354, 43002, 11573, 7640, 34949, 76473, 33126, 11573, 69933, 7640, 34949, 33126, 52635, 75964, 11573, 60294, 42354, 11573, 36510, 34949, 75964, 45234, 42354, 35674, 33126, 47541, 11573, 43002, 75964, 11573, 69584, 12264, 24253]
decoded_message = Decode(77851, 36095, encoded_message)
print(decoded_message)

# Answer to Question - Connor Brown
string = "This is too deep a question for me to answer with one thing. I would hope aliens would discover our capacity to do both good and evil - the human paradox."
encoded_message = Encode(77851, 47, string)
print()
print("Cipher =", encoded_message)

print()
#################################

# Question - by Linda M.
encoded_message = [74952, 94157, 141543, 73103, 5165, 409, 43504, 146715, 96940, 5165, 94157, 117516, 43504, 5165, 117516, 110969, 43504, 96940, 409, 20539, 141543, 80556, 5165, 20539, 117516, 5165, 11385, 117516, 110969, 5165, 94112, 83619, 141543, 35329, 141543, 83619, 5165, 409, 138638, 141543, 5165, 138638, 83619, 141543, 106642, 147401, 5165, 117516, 83619, 5165, 106642, 5165, 94112, 117516, 94112, 96940, 409, 138638, 80559, 141543, 52289, 5165, 74952, 94157, 106642, 43504, 146715, 96940, 5165, 11385, 117516, 110969, 83619, 5165, 35329, 106642, 41334, 117516, 83619, 409, 43504, 141543, 52289]
decoded_message = Decode(153193, 91433, encoded_message)
print(decoded_message)

# Answer to Question - Connor Brown
string = "I'm weird and like ice cream when it's cold outside. But when it's hot, I like popsicles and shaved ice. Flavor depends on my mood."
encoded_message = Encode(153193, 5, string)
print()
print("Cipher =", encoded_message)

print()


If you could have a superpower, what would it be?

Cipher = [3255, 2178, 1116, 4621, 2178, 3258, 3641, 3099, 485, 4815, 2178, 2209, 245, 3641, 882, 245, 245, 2153, 2178, 3099, 2153, 5370, 523, 4621, 523, 3641, 245, 5439, 2178, 962, 5319, 151, 3258, 523, 485, 1116, 5370, 2178, 2497, 245, 5586, 245, 2153, 245, 2497, 1116, 3641, 523, 3995, 2153, 2178, 1116, 2153, 5439, 2178, 5370, 523, 2153, 245, 2221, 3995, 3531, 2221, 3258, 523, 5586, 5319, 3641, 2178, 3641, 245, 5370, 245, 962, 3995, 2497, 3641, 1116, 3641, 523, 3995, 2153, 3383, 2178, 3255, 2178, 882, 1116, 2153, 3641, 2178, 3641, 3995, 2178, 2209, 3995, 3641, 5319, 2178, 2209, 245, 2178, 5319, 245, 1116, 5370, 3641, 5319, 151, 2178, 1116, 2153, 5439, 2178, 5586, 3995, 2178, 1116, 5370, 5370, 2178, 4815, 523, 2153, 5439, 3258, 2178, 3995, 3531, 2178, 962, 5370, 1116, 485, 245, 3258, 2621]

Should civilization collapse, what piece of media (book, movie, album, etc.) would you like aliens to discover us by?

Cipher = [18147, 24171, 349

### 7. Project Narrative

My process for this project was a solid combination of techniques. A few weeks ago, I was new to RSA and the mathematical techniques and algorithms involved in its execution. For me, learning those first was my first priority. I am sometimes slow to assimilate new information, so I wasn't really sure what to plan ahead for. It's hard to plan for something that I have never done before. The Mastery Workbooks definitely helped - they gave me work to do on the project relevant to that week's curriculum and encouraged me to plan ahead as best I could. In other words, I mostly planned ahead, telling myself when I would be getting certain parts of the project done, and then "winging" the rest, because that's the nature of doing new things. I am methodical whenever possible and adaptable whenever necessary.

It was satisfying to get encryption and decryption working. I was a little unsure of whether or not my code would work, but I tried to remember that this project is designed to facilitate success as long as you are diligent in the coursework. I was initially having some trouble encrypting and decrypting messages. I would input a message, then encrypt/decrypt it, and end up with a string of nonsense characters. I think this was because I did not initially write a Python script to keep the order of operations correct. Once I did that, making sure I was passing the correct p, q, e, and d values into my functions, I had more success. Overall, the code exchange went well. I was immediately able to decrypt other students' posts using their private keys, and then writing responses using the public ones. I always made to sure to double check that my responses were decrypting into readable messages. The repeated testing and checking was what took the most time. Sometimes the decryption would not work, and I think that was related to a bug in my Extended Euclidean Algorithm that I fixed soon after.

The most challenging part of this project was having to learn large amounts of new theory, and nearly immediately put it into practice. It was difficult to plan out something so new and fast-paced. Even when looking ahead, I didn't always understand what I was going to be doing, so it sometimes felt like being blindfolded and led by the hand through an unknown place. I just tried to be diligent in my studies and "trust the process".

Overall, I think this project was well-designed. It used material exactly from the course, and followed the curriculum timeline. One helpful resource was the Python Docs; that was where I found the "secrets" module, which I used to generate public keys with high-quality randomness. Another was GeeksForGeeks, which had some useful tips on coding and modules.

My Best Mistake was a bug in my code that caused my decryption function to periodically not work and return strings of nonsense characters. I thought I had found the issue: sometimes my d value was not the correct modular inverse of e mod phi. I was up late one night trying to figure out why, but got totally stumped. After office hours, I tried to check that e and d were relatively prime, but I was still getting junk values. Sometimes the modular inverse was correct, but still ended up giving me junk decryptions. My next thought was that the problem was inside of my Extended Euclidean Algorithm function, because that was where I was doing something different with my d value.  

In order to make EEA work, I swapped the inputs so that a would always be greater than b in the equation a mod b. I made sure to swap the resulting s1 and t1 values to reflect this. I also made sure to add the initial value of a, a0 to s1 until it was positive, so that my decryption would work later. What I failed to notice was that depending on the order of the inputs, it was necessary to add either a0 OR b0 to s1. I noticed this when I printed out the linear combinations of EEA(1012, 27) and EEA(27, 1012). To fix the issue, I added a conditional to the loop for updating s1 to make it positive, ensuring that the correct amount (a0 or b0) was added. This solved the problem immediately. I was deeply frustrated by this problem for several days, so I was really happy to figure it out. Lesson learned: when swapping numbers, CHECK EVERYTHING CAREFULLY.


### 8. Code Breaking

Basic code breaking is done by using a brute force algorithm to factor n to get p and q values, then reverse engineering the private key d and using it to decrypt a message.


**Brute Force Factorization**

In [78]:
def factorize(n):
    # n is a number, return the smallest factor of n
    for i in range(2, n):
        if n % i == 0:
            return i
    return False

**How Code Breaking Works**

Code breaking is pretty straightforward. First, get the public key and encrypted_message given by the writer of the code:

In [79]:
n, e = (781, 9021)   # public key
encrypted_message = [72, 321, 174, 174, 144, 539, 329, 604, 144, 444, 174, 221, 209]

Next, factor the n value to get the p and q values using the factorize() function and arithmetic:

In [80]:
p = factorize(n)
q = n // p

Using these p and q values, as well as the provided public key e, determine the private key using Find_Private_Key_d():

In [81]:
d = Find_Private_Key_d(e, p, q)

Lastly, using p, q, and the calculated private key d, decode the encrypted message using Decode():

In [82]:
decoded_text = Decode(n, d, encrypted_message)
print(decoded_text)

Hello, World!


After brute forcing an n value with 17 digits, I noticed that it takes a REALLY long time to brute force larger values of n, especially after n exceeds 14 digits. The p and q values are calculated fairly quickly, but the decryption is quite slow. If large values of n are used, RSA is very secure, because it takes a VERY LONG TIME to process the large numbers involved, if they can be processed at all. However, my implementation is not very secure, because it uses relatively small numbers, which can be cracked easily.

### 9. Code Breaking: Complete Examples


**Code Breaking Example**

I will be breaking Justin L's code. The public key and cipher are provided:

In [83]:
n, e = 6883612823, 67 # Public Key

Cipher =  [6228687168, 2479805619, 3810606779, 6374462197, 1970299466, 4967789840, 594058649, 1175235167, 1614932420, 2479805619, 3810606779, 1614932420, 6024407370, 1463405190, 2660112545, 1970299466, 5064114676, 2342712351, 594058649, 3868547149, 3810606779, 6374462197, 1614932420, 5810967685, 5064114676, 1614932420, 5810967685, 1463405190, 1175235167, 1175235167, 4967789840, 6374462197, 1463405190, 132705964, 1614932420, 5962446038, 6471172030, 4967789840, 594058649, 1614932420, 3868547149, 1175235167, 1614932420, 5064114676, 2479805619, 3146077563, 1970299466, 1614932420, 6735133346, 4967789840, 4722640704, 2479805619, 1970299466, 3868547149, 594058649, 1463405190, 1614932420, 5810967685, 2479805619, 4722640704, 3868547149, 1463405190, 1172750983]

Factor the n value to get the p and q values:

In [84]:
p = factorize(n)
q = n // p

Using these p and q values, as well as the provided public key e, determine the private key using Find_Private_Key_d():

In [85]:
d = Find_Private_Key_d(e, p, q)

Decode the encrypted message using n and private key d:

In [86]:
decoded_text = Decode(n, d, Cipher)
print(decoded_text)

Congrats on decrypting my message! What is your favorite movie?


**Decode and Respond**

The following Code_Breaker() function is used to automate the steps used to "brute force" decrypt messages:

In [87]:
def Code_Breaker(n_val, e_val, cipher_text):
    """
    Decodes messages using given public key and brute force factorization algorithm.
    """
    p = factorize(n_val)
    q = n_val // p
    d = Find_Private_Key_d(e_val, p, q)
    decoded_text = Decode(n_val, d, cipher_text)
    
    return decoded_text

In [88]:
print()

# Message - by Rachel C.
n, e = 130177, 13 #Public Key
encoded_message = [85308, 48594, 15927, 71285, 61037, 15927, 767, 23406, 71285, 23406, 48594, 12676, 37298, 100459, 71285, 90170, 61037, 38946, 38783, 23406, 71285, 90170, 71285, 113492, 38946, 38946, 86792, 15927, 90170, 37298, 71285, 12676, 767, 71285, 15927, 121493, 15927, 37298, 71285, 12676, 84628, 71285, 29228, 38946, 38783, 71285, 90170, 110238, 15927, 71285, 81798, 110238, 38946, 37298, 100459, 126494, 71285, 29228, 38946, 38783, 71285, 90170, 110238, 15927, 71285, 38946, 37298, 86792, 29228, 71285, 38946, 84628, 84628, 71285, 61037, 29228, 71285, 90170, 71285, 61037, 12676, 23406, 10510]
print(Code_Breaker(n, e, encoded_message))

# Response - Connor Brown
string = "There are 10 types of people in the world: those who understand binary, and those who don't."
encoded_message = Encode(130177, 13, string)
print("Cipher =", encoded_message)
print(Code_Breaker(n, e, encoded_message))

print()
#################################

# Message - by Faisal S.
n, e = 559 ,5 #Public Key
encoded_message = [417, 150, 457, 127, 24, 52, 457, 367, 511, 203, 24, 367, 511, 367, 457, 12, 234, 417, 20, 44, 457, 127, 24, 52, 457, 24, 448, 511, 457, 447, 511, 457, 426, 52, 314, 203, 234, 362]
print(Code_Breaker(n, e, encoded_message))

# Response - Connor Brown
string = "Can we go halfsies?"
encoded_message = Encode(559, 5, string)
print("Cipher =", encoded_message)
print(Code_Breaker(n, e, encoded_message))

print()
#################################

# Message - by Linda M.
n, e =  869, 7 #Public Key
encoded_message = [317, 19, 209, 35, 637, 716, 88, 117, 306, 637, 36, 763, 88, 89, 840, 192, 19, 63, 63, 117, 88, 637, 716, 192, 637, 63, 253, 216, 117, 89, 145]
print(Code_Breaker(n, e, encoded_message))

# Response - Connor Brown
string = "I handwrite notes, but type everything else."
encoded_message = Encode(869, 7, string)
print("Cipher =", encoded_message)
print(Code_Breaker(n, e, encoded_message))

print()


The best thing about a Boolean is even if you are wrong, you are only off by a bit.
Cipher = [85308, 48594, 15927, 110238, 15927, 71285, 90170, 110238, 15927, 71285, 81670, 115640, 71285, 23406, 29228, 114803, 15927, 767, 71285, 38946, 84628, 71285, 114803, 15927, 38946, 114803, 86792, 15927, 71285, 12676, 37298, 71285, 23406, 48594, 15927, 71285, 81798, 38946, 110238, 86792, 38831, 14527, 71285, 23406, 48594, 38946, 767, 15927, 71285, 81798, 48594, 38946, 71285, 38783, 37298, 38831, 15927, 110238, 767, 23406, 90170, 37298, 38831, 71285, 61037, 12676, 37298, 90170, 110238, 29228, 126494, 71285, 90170, 37298, 38831, 71285, 23406, 48594, 38946, 767, 15927, 71285, 81798, 48594, 38946, 71285, 38831, 38946, 37298, 92171, 23406, 10510]
There are 10 types of people in the world: those who understand binary, and those who don't.

if you decoded this, you owe me lunch!
Cipher = [357, 145, 314, 457, 448, 511, 457, 298, 24, 457, 234, 145, 426, 150, 20, 417, 511, 20, 241]
Can we go halfsies?

Pic


### 10. Custom Feature

For my custom feature, I went with a main() function. It is used to automate the encryption and decryption process. While it's not my first time building one, I did this one entirely myself with little-to-no assistance from programming resources such as GeeksforGeeks. It uses the RSA package I built earlier in the project, as well as helper functions I built afterwards.

It displays a menu with four options: Get Keys, Encode a Message, Decode a Message, and Exit. Get Keys prompts the user for two prime numbers, which are used to calculate the public and private keys needed to encode a message. Encode a Message prompts the user for the public keys and a string message, which are encoded into a list of integers and displayed to the user. Decode a Message does the opposite. It prompts the user for the private key and a list of encoded integers, which are then converted back into a comprehensible string message and displayed to the user. Exit ends the program. There is also a catchall for the case that the user selects an unavailable option.

It was simple and straightforward to implement, and makes it much easier to encode and decode messages. Yay computers!

In [89]:
def FME(b, n, m):
    """
    Using the fast modular exponentiation algorithm, returns b**n mod m.
    """
    
    # initial values
    result = 1
    square = b
    
    while (n > 0):
        # convert n to binary, start from least significant digit
        k = n % 2 
        # add to result if binary digit is 1
        if k == 1:
            result = (result * square) % m
        # advance exponent base
        square = (square * square) % m
        # advance to next digit in n while reducing n
        n //= 2
        
    return result


def Euclidean_Alg(a, b):
    """
    1. Calculate the Greatest Common Divisor of a and b.  
    2. This version should have only positive inputs and outputs.
    3. The function must return a single integer ('x') which is
    the gcd of a and b.
    """
    
    # check inputs for correct type and value
    if (type(a) != int) or (type(b) != int):
        raise TypeError("Inputs must be positive integers.")
    if (a <= 0) or (b <= 0):
        raise ValueError("Inputs must be positive integers.")
    
    # swap inputs if a is not greater than b
    if (b > a):
        temp = a
        a = b
        b = temp
    
    # calculate gcd
    while (b > 0):
        # find remainder of a / b
        k = a % b
        # advance a to next smallest number
        a = b
        # advance b to remainder
        b = k
    
    # gcd equal to final value of a
    x = a
    
    return x


def EEA(a, b):
    """
    This function returns: 
    1. the GCD of a, b 
    2. Bezout's coefficients (s1, t1)
    
    Inputs are positive integers.
    
    a must be greater than b.
    """
        
    # check inputs for correct type and value
    if (type(a) != int) or (type(b) != int):
        raise TypeError("Inputs must be positive integers.")
    if (a <= 0) or (b <= 0):
        raise ValueError("Inputs must be positive integers.")
    
    # swap inputs if a is not greater than b
    isSwapped = b > a
    if (isSwapped):
        temp = a
        a = b
        b = temp
        
    # initial values of a and b
    a0 = a
    b0 = b
    
    # initial values of Bezout coefficients
    s1, t1 = (1, 0)
    s2, t2 = (0, 1)
    
    # calculate gcd and Bezout coefficients
    while (b > 0):
        # update a and b using new Bezout coefficients
        a = (s1 * a0) + (t1 * b0)
        b = (s2 * a0) + (t2 * b0)
        
        # find remainder of a / b
        k = a % b
        # find integer quotient of a / b
        q = a // b
        
        # advance a to next smallest number
        a = b
        # advance b to remainder
        b = k
        
        # calculate Bezout coefficients for next smallest number
        s1_p, t1_p = (s2, t2)
        s2_p, t2_p = (s1 - (q * s2), t1 - (q * t2))
        
        # update actual Bezout coefficients
        s1, t1 = (s1_p, t1_p)
        s2, t2 = (s2_p, t2_p)
        
    # gcd equal to final value of a
    gcd = a
    
    # reflect original order of a and b
    if isSwapped:
        s1_final = t1
        t1_final = s1
    else:
        s1_final = s1
        t1_final = t1
        
    # ensure that s1_final is positive using modular inverse trick
    while (s1_final < 0):
        if isSwapped:
            s1_final += a0
        else:
            s1_final += b0
        t1_final = (gcd - (s1_final * a0)) // b0
        
    
    return gcd, (s1_final, t1_final)


def Find_Public_Key_e(p, q):
    """
    Takes in 2 primes p and q.
    
    Returns 2 elements as follows:
    public key: n
    public key: e
    """
    
    # for getting quality random numbers
    from secrets import randbelow
    
    # calculate n and phi(n)
    n = p * q
    phi_n = (p - 1) * (q - 1)
    
    # calculate initial value for e
    e = randbelow(10000)
    
    # update e value until required conditions are met
    while (Euclidean_Alg(phi_n, e) != 1 or e == p or e == q):
        e = randbelow(10000)
        
    return (n, e)


def Find_Private_Key_d(e, p, q):
    """
    Returns the decryption exponent d, such that d is the modular inverse of e. 
    
    This will use the Extended Euclidean Algorithm.
    
    This function should return the following:
    d: the decryption component.
    """
    
    # calculate n and phi(n)
    n = p * q
    phi_n = (p - 1) * (q - 1)
    
    # d is modular inverse of e mod phi_n - calculate using EEA of (e, phi_n)
    gcd, (s1, t1) = EEA(e, phi_n)
    
    # d is equal to Bezout coefficient s1
    d = s1
    
    return d


def Convert_Text(_string):
    """
    Takes in a string and returns the corresponding
    standard list of integers (ascii) for each letter in the string.
    """
    integer_list = []
    
    # add converted characters to list
    for char in _string:
        integer_list.append(ord(char))
        
    return integer_list


def Convert_Num(_list):
    """
    Takes in a list of integers (ascii)
    and outputs the corresponding string.
    """
    _string = ''
    
    # update string with converted integers
    for i in _list:
        _string += chr(i)
        
    return _string


def Encode(n, e, message):
    """
    Encodes message using by converting message to integer list
    and then uses n, e, and FME to alter values.
    
    Returns cipher_text (list)
    """
    
    # get unencoded list
    int_list = Convert_Text(message)
    
    # encoded list
    cipher_text = []
    
    # encode items in int_list using FME, add to cipher_text
    for m in int_list:
        m_encrypted = FME(m, e, n)
        cipher_text.append(m_encrypted)
    
    return cipher_text


def Decode(n, d, cipher_text):
    """
    Takes in a list of encoded integers. Decodes message using
    n, d, and FME, then converts resulting number list to a string.
    
    Returns: message (string) 
    """
    
    message = ''
    
    # decrypt cipher text list using FME
    decrypted_cipher_text = [FME(c, d, n) for c in cipher_text]
    
    # convert decrypted list to string
    message = Convert_Num(decrypted_cipher_text)

    return message


##########################
##### CUSTOM FEATURE #####
##########################


def menu():
    """
    Displays menu for command line interface.
    """
    
    print("==========================")
    print(f"{'RSA PROJECT':^26}")
    print(f"{'Discrete Structures':^26}")
    print(f"{'Connor Brown':^26}")
    print(f"{'[ID: XXXXXXXXX]':^26}")
    print("==========================")
    print()
    print("Options:")
    print("1. Get Keys")
    print("2. Encode a Message")
    print("3. Decode a Message")
    print("4. Exit")

    
def get_keys():
    """
    Takes user input for two prime numbers p and q and
    outputs public and private keys.
    """
    
    # get user input
    p = int(input("Enter first prime number 'p': "))
    q = int(input("Enter second prime number 'q': "))
    
    # get public and private key values
    n, e = Find_Public_Key_e(p, q)
    d = Find_Private_Key_d(e, p, q)
    
    # display results
    print(f"Your public keys (n, e) are: ({n}, {e})")
    print(f"Your private key 'd' is: {d}")

    
def encode_message():
    """
    Takes user input for public keys (n, e) and a message,
    then prints the encrypted message as a list of numbers.
    """
    
    # get user input - public keys and message to encrypt
    n = int(input("Enter public key 'n': "))
    e = int(input("Enter public key 'e': "))
    message = input("Enter a message to encrypt, then press ENTER: ")
    
    # encrypt message using keys
    ciphertext = Encode(n, e, message)
    
    # display encrypted message
    print()
    print(f"Ciphertext: {ciphertext}")

    
def decode_message():
    """
    Takes user input for public key 'n', private key 'd', and
    an encrypted message list, then prints the decrypted message
    as a string.
    """
    # get user input - public key n, private key d and ciphertext list
    n = int(input("Enter public key 'n': "))
    d = int(input("Enter private key 'd': "))
    ciphertext_str = input("Enter ciphertext as a list (e.g. [23, 14]): ")
    
    # convert ciphertext string input into Python list of integers
    ciphertext = [int(i) for i in ciphertext_str.strip("[]").split(",")]
    
    # decrypt cipher using keys
    message = Decode(n, d, ciphertext)
    
    # display decrypted message
    print()
    print("The decoded message is:")
    print(message)
    

def main():
    """
    Creates a command line interface (via helper functions) that allows uses to encrypt and decrypt data using RSA system.
    """
    menu()
    selection = int(input("Make a selection: "))
    print()
    # display options until user quits (inputs '4')
    while (selection != 4):
        if selection == 1:
            get_keys()
        elif selection == 2:
            encode_message()
        elif selection == 3:
            decode_message()
        else:
            print("Invalid selection. Please try again.")
        
        print()
        next_selection = input("Press ENTER to continue")
        print()
        menu()
        selection = int(input("Make a selection:"))
        print()
     
    # selection == 4: exit program
    print("Goodbye.")
    

# run main as script
if __name__ == '__main__':
    main()

       RSA PROJECT        
   Discrete Structures    
       Connor Brown       
     [ID: XXXXXXXXX]      

Options:
1. Get Keys
2. Encode a Message
3. Decode a Message
4. Exit


ValueError: invalid literal for int() with base 10: ''

**Custom Feature Demonstration:**

The following block is an example of using my custom main() function, in case the program does not work:

## Self grading of custom feature ##

For my custom feature score, I will give myself an 8 out of 10. I did not push myself very hard on it, but I did a good job, commented effectively, and demonstrated my mastery of Python skills. 

I built a main() function to help automate the encryption and decryption process. While it's not my first time building one, I did this one entirely myself with little-to-no assistance from programming resources such as GeeksforGeeks. It uses helper functions and the RSA package I built earlier in the project. Learning about and implementing RSA was something completely new to me, so I wanted to do a custom feature that was a little more familiar.